In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/rithanya/darri_pipeline/STEREO_TO_PC

/content/drive/MyDrive/rithanya/darri_pipeline/STEREO_TO_PC


In [4]:
#@title GET THE CALIBERATION DATA, WE FOCUS ON Q MATRIX
import numpy as np
import cv2

def parse_calibration_and_compute_Q(filepath):
    """Parse KITTI calibration file and compute Q matrix."""
    data = {}
    with open(filepath, 'r') as f:
        for line in f.readlines():
            if ':' in line:
                key, value = line.split(':')
                data[key] = np.array([float(x) for x in value.split()])

    # Extract camera parameters
    P2 = data['P2'].reshape(3, 4)
    P3 = data['P3'].reshape(3, 4)
    f = P2[0, 0]
    cx, cy = P2[0, 2], P2[1, 2]
    Tx = P3[0, 3] / (-f)

    # Compute Q matrix
    Q = np.zeros((4, 4))
    Q[0, 0] = Q[1, 1] = 1
    Q[0, 3] = -cx
    Q[1, 3] = -cy
    Q[2, 3] = f
    Q[3, 2] = -1 / Tx
    Q[3, 3] = (cx - P3[0, 2]) / Tx  # cx' - cx

    return data, Q

# Example usage
calib_data, Q_matrix = parse_calibration_and_compute_Q('/content/drive/MyDrive/rithanya/DARRI_PSUEDO_LIDAR/deep_learning_disparity/0000.txt')
print("Calibration data:")
print(calib_data)
print("\nQ matrix:")
print(Q_matrix)


Calibration data:
{'P0': array([721.5377,   0.    , 609.5593,   0.    ,   0.    , 721.5377,
       172.854 ,   0.    ,   0.    ,   0.    ,   1.    ,   0.    ]), 'P1': array([ 721.5377,    0.    ,  609.5593, -387.5744,    0.    ,  721.5377,
        172.854 ,    0.    ,    0.    ,    0.    ,    1.    ,    0.    ]), 'P2': array([7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01,
       0.000000e+00, 7.215377e+02, 1.728540e+02, 2.163791e-01,
       0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03]), 'P3': array([ 7.215377e+02,  0.000000e+00,  6.095593e+02, -3.395242e+02,
        0.000000e+00,  7.215377e+02,  1.728540e+02,  2.199936e+00,
        0.000000e+00,  0.000000e+00,  1.000000e+00,  2.729905e-03])}

Q matrix:
[[   1.            0.            0.         -609.5593    ]
 [   0.            1.            0.         -172.854     ]
 [   0.            0.            0.          721.5377    ]
 [   0.            0.           -2.12514366    0.        ]]


In [2]:
%cd /content/drive/MyDrive/rithanya/darri_pipeline/STEREO_TO_PC

/content/drive/MyDrive/rithanya/darri_pipeline/STEREO_TO_PC


In [5]:
# disparity_video_processing.py

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from PSMnet_for_disparity import PSMNet, test_PSM

# Initialize CUDA, model, and other variables
cuda_present = torch.cuda.is_available()
maxdisp = 192
# Path for left and right images
left_folder_path = "/content/drive/MyDrive/rithanya/darri_pipeline/DARRI_PIPE_CHECK/LEFT"
rght_folder_path = "/content/drive/MyDrive/rithanya/darri_pipeline/DARRI_PIPE_CHECK/RIGHT"

# Pretrained model path
pretrained_model_path = "/content/drive/MyDrive/rithanya/DARRI_PSUEDO_LIDAR/deep_learning_disparity/pretrained_model_KITTI2012.tar raw=true"

# Output folders
output_folder = "/content/drive/MyDrive/rithanya/darri_pipeline/DARRI_PIPE_CHECK/OUTPUT/disparity2/"
disp_npy_op = "/content/drive/MyDrive/rithanya/darri_pipeline/DARRI_PIPE_CHECK/OUTPUT/disp_numpy2/"
ply_output_folder = "/content/drive/MyDrive/rithanya/darri_pipeline/DARRI_PIPE_CHECK/OUTPUT/pointcloud_op2/"

model = PSMNet(maxdisp)

if cuda_present:
    torch.cuda.manual_seed(1)
    model = nn.DataParallel(model)
    model.cuda()

state_dict = torch.load(pretrained_model_path)
model.load_state_dict(state_dict['state_dict'])

# Ensure output directories exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(disp_npy_op, exist_ok=True)
os.makedirs(ply_output_folder, exist_ok=True)

# Get sorted lists of left and right image file names
left_images = sorted([img for img in os.listdir(left_folder_path)])
rght_images = sorted([img for img in os.listdir(rght_folder_path)])

# Process each image pair
for left_image_name, rght_image_name in zip(left_images, rght_images):
    left_image_path = os.path.join(left_folder_path, left_image_name)
    rght_image_path = os.path.join(rght_folder_path, rght_image_name)

    disp_img = test_PSM(model, left_image_path, rght_image_path, cuda_present)
    disp = np.array(disp_img, dtype=np.float64)
    disp = cv2.normalize(disp, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    np.save(os.path.join(disp_npy_op, f"disparity_{left_image_name.split('.')[0]}.npy"), disp)
    cv2.imwrite(os.path.join(output_folder, f"disparity_{left_image_name.split('.')[0]}.png"), disp)

    # Generate Point Cloud from Disparity Map
    colors = cv2.cvtColor(cv2.imread(left_image_path), cv2.COLOR_BGR2RGB)
    mask_map = disp > disp.min()
    output_points = cv2.reprojectImageTo3D(disp, Q_matrix)[mask_map]
    output_colors = colors[mask_map]

    def create_point_cloud_file(vertices, colors, filename):
        colors = colors.reshape(-1, 3)
        vertices = np.hstack([vertices.reshape(-1, 3), colors])
        ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
        with open(filename, 'w') as f:
            f.write(ply_header % dict(vert_num=len(vertices)))
            np.savetxt(f, vertices, '%f %f %f %d %d %d')

    output_file = os.path.join(ply_output_folder, f'pointCloud_{left_image_name.split(".")[0]}.ply')
    create_point_cloud_file(output_points, output_colors, output_file)


In [1]:
#@title visualize "a" point_cloud USING OPEN3D

import open3d as o3d
import numpy as np

# Replace 'path_to_your_ply_file.ply' with the path to your PLY file
ply_file_path = '/content/drive/MyDrive/rithanya/darri_pipeline/DARRI_PIPE_CHECK/OUTPUT/pointcloud_op/pointCloud_000029L.ply'

# Load your own PLY point cloud
pcd = o3d.io.read_point_cloud(ply_file_path)

# Print the point cloud
print(pcd)
print(np.asarray(pcd.points))

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


PointCloud with 465749 points.
[[ 32.557705   9.232456 -38.538681]
 [ 32.504292   9.232456 -38.538681]
 [ 39.29599   11.179928 -46.667934]
 ...
 [ -1.190061  -0.3803    -1.364186]
 [ -1.186694  -0.378622  -1.358168]
 [ -1.176857  -0.374889  -1.344776]]
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
#@title Visualize a "video" pointclouds USING MAYAVI

import os
import open3d as o3d
import numpy as np
import mayavi.mlab as mlab

def visualize_point_clouds(input_dir, output_dir):
    def rotate_x(points, angle):
        theta = np.radians(angle)
        c, s = np.cos(theta), np.sin(theta)
        R = np.array([
            [1, 0, 0],
            [0, c, -s],
            [0, s, c]
        ])
        return np.dot(points, R.T)

    def rotate_z(points, angle):
        theta = np.radians(angle)
        c, s = np.cos(theta), np.sin(theta)
        R = np.array([
            [c, -s, 0],
            [s, c, 0],
            [0, 0, 1]
        ])
        return np.dot(points, R.T)

    input_files = os.listdir(input_dir)
    os.makedirs(output_dir, exist_ok=True)

    for i, file_nm in enumerate(input_files):
        pcd = o3d.io.read_point_cloud(os.path.join(input_dir, file_nm))
        pcloud = np.array(pcd.points)
        colors = np.array(pcd.colors)
        rgba_colors = np.ones((colors.shape[0], 4))
        rgba_colors[:, :3] = colors

        pcloud_rotated = pcloud.copy()
        pcloud_rotated[:, 2] = -pcloud_rotated[:, 2]
        pcloud_rotated[:, 0] = -pcloud_rotated[:, 0]
        pcloud_fig = mlab.figure(bgcolor=(0, 0, 0), size=(1280, 720))
        mlab.clf()
        plot = mlab.points3d(pcloud_rotated[:, 0], pcloud_rotated[:, 1], pcloud_rotated[:, 2], np.arange(len(pcloud_rotated)), mode='point', figure=pcloud_fig)
        plot.module_manager.scalar_lut_manager.lut.table = (rgba_colors * 255).astype(np.uint8)

        mlab.view(azimuth=270, distance=10, elevation=10, focalpoint=np.mean(pcloud_rotated, axis=0))
        mlab.savefig(filename=os.path.join(output_dir, f'pointcloud_{i:02d}.png'))
        mlab.close(all=True)

# Usage
input_dir = "pointcloud_output_folder0006-20240515T141019Z-001/pointcloud_output_folder0006/"
output_dir = "pointcloud_img0006/"
visualize_point_clouds(input_dir, output_dir)


In [ ]:
#@title after checking the point_cloud_representations, you can create a video
import os
import cv2
from tqdm import tqdm

def create_point_cloud_video(input_dir, output_video_path, fps=1, codec='VP80'):
    """
    Creates a video from images stored in a directory.

    Parameters:
    - input_dir (str): Path to the directory containing point-cloud images.
    - output_video_path (str): Path to the output video file.
    - fps (int): Frames per second for the video. Default is 1.
    - codec (str): Codec to be used for video. Default is 'VP80'.
    """
    fourcc = cv2.VideoWriter_fourcc(*codec)
    writer = None

    for file in tqdm(os.listdir(input_dir)):
        img_path = os.path.join(input_dir, file)

        img = cv2.imread(img_path)
        if img is None:
            continue

        if writer is None:
            writer = cv2.VideoWriter(output_video_path, fourcc, fps, (img.shape[1], img.shape[0]), True)

        if writer is not None:
            writer.write(img)

    if writer is not None:
        writer.release()

input_dir = "normal/0000/0000/"
output_video_path = "0000_video.webm"
create_point_cloud_video(input_dir, output_video_path)
